In [3]:
import pathlib
import sys

sys.path.append("../")

local_path = pathlib.Path().resolve()

from typing import List, Optional, Dict, Tuple
from scipy.optimize import minimize
from collections import deque

import pandas as pd
import rust_statespace as rs
import numpy as np
import matplotlib.pyplot as plt

from statespace.utils import _initiate_variables, _map_vector_to_matrices, _get_nan_positions, _remove_nan_tensor, _remove_inf_tensor, _get_bounds, read_target_from_path
from statespace.StateSpacingKalman import KalmanV1, KalmanV2, KalmanV3
from statespace.StateSpacingProtocols import KalmanProtocol, MathProtocol
from statespace.StateSpacingClasses import LinearGaussianModel_v2_numpy as LGM
from statespace.StateSpacingClasses import LLM_v1_numpy as LLM

In [4]:
rs.kalman_filter

<function rust_statespace.rust_statespace.kalman_filter>

# Test different implementations of Kalman protocol implementations

In [8]:
KalmanV1 = KalmanV1()
print(KalmanV1.ssmath)

<class 'statespace.StateSpacingMath.StateSpaceMathNumpyV1'>


In [9]:
KalmanV2 = KalmanV2()
print(KalmanV2.ssmath)

<class 'statespace.StateSpacingMath.StateSpaceMathNumpyV2'>


In [10]:
KalmanV3 = KalmanV3()
print(KalmanV3.ssmath)

<class 'statespace.StateSpacingMath.StateSpaceMathNumpyV3'>


## Load data for testing 

In [9]:
dtype=np.float64
base_dir = pathlib.Path().resolve().parent
data_dir = base_dir / 'data'
llt_path = data_dir / 'llt.txt'

# get data into shape [s, 1, n]
y = read_target_from_path(llt_path, dtype)
#y[:, :, 60:80] = np.nan

# Declare State Matrices Local linear Trend Model -> use [[]] for extra dimension
T = np.array([[1, 1], [0, 1]]).astype(dtype)
Z = np.array([[1, 0]]).astype(dtype)
R = np.array([[1, 0], [0, 1]]).astype(dtype)
Q = np.array([[1, 0], [0, 1]]).astype(dtype)
H = np.array([[1]]).astype(dtype)
diffuse = True

a = np.array([[1], [0]]).astype(dtype)
P = np.array([[5, 6], [2, 3]]).astype(dtype)

n=100
shapes = {'n': n, 'p': 2, 's': 1}

T = np.repeat(T[:, :, None], n, axis=2)
Z = np.repeat(Z[:, :, None], n, axis=2)
R = np.repeat(R[:, :, None], n, axis=2)
Q = np.repeat(Q[:, :, None], n, axis=2)
H = np.repeat(H[:, :, None], n, axis=2)

## Kalman Filter Iteration

In [12]:
%timeit KalmanV1._kalman_step(T[:, :, 10], Z[:, :, 10], R[:, :, 10], Q[:, :, 10], H[:, :, 10], a, P, y[0, 0, 10])

18.9 µs ± 92.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
%timeit KalmanV2._kalman_step(T[:, :, 10], Z[:, :, 10], R[:, :, 10], Q[:, :, 10], H[:, :, 10], a, P, y[0, 0, 10])

18.9 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
%timeit KalmanV3._kalman_step(T[:, :, 10], Z[:, :, 10], R[:, :, 10], Q[:, :, 10], H[:, :, 10], a, P, y[0, 0, 10])

21 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Kalman Filter

In [15]:
%timeit KalmanV1.kalman_filter(T, Z, R, Q, H, y, diffuse, shapes)

2.1 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit KalmanV2.kalman_filter(T, Z, R, Q, H, y, diffuse, shapes)

2.09 ms ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit KalmanV3.kalman_filter(T, Z, R, Q, H, y, diffuse, shapes)

2.3 ms ± 8.03 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit rs.kalman_filter(T[:, :, 10], H[:, :, 10], Q[:, :, 10], Z[:, :, 10], R[:, :, 10], y)

233 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
